# FourierGNN prediction model - capstone analysis

FourierGNN https://github.com/aikunyi/FourierGNN

I have forked the FourierGNN and made some minor modifications to adapt it for our use case. Moreover, some of the functions from FourierGNN are used in the below notebook to support modelling.


Uses FourierGNN to predict 1-step ahead values of the multivariate time series. 

A Fully Connected Neural Network is then used to predict the log returns value from the FourierGNN predictions


In [29]:
from dataclasses import dataclass
import time

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utils.utils import save_model, load_model, evaluate
from data.data_loader import Dataset_Capstone
from model.FourierGNN import FGN

## Data preparation

Prepare 2 datasets for use in FourierGNN model.
- Dataset 1 includes 2xx features
- Dataset 2 uses a selected set of features

Input data has already been transformed towards stationarity through Yeo-Johnson and differencing. The dataset starts post COVID

In [ ]:
df = pd.read_csv('filtered_data.csv')
df.shape

Subset on post-COVID period. 

The COVID period appeared to reduce model performance in testing. Given it is not expected to be representative of the models prediction regime, this model will only consider post-Covid data

In [ ]:
# Showing COVID spike
df[['FACTSET_SP_500_Close']].plot()

In [ ]:
# Index soon after COVID spike
covid_idx = 370

In [ ]:
# Evidencing that COVID spike is not present in the training data
df[['FACTSET_SP_500_Close']][covid_idx:].plot()

In [ ]:
# Updating df to only contain data after COVID spike
df = df[covid_idx:]

Create two testing datasets, one containing all available variables, and the other containing a reduced set of columns. The features selected for the subsequent dataset were chosen due to their high feature importances in an XGBoost model and a Lasso model

In [ ]:
df_1 = df.copy()

Select key features

In [ ]:
lasso_top_features = [
    'FACTSET_P_PER_B_LTM',
    'FACTSET_P_PER_CF_LTM',
    'FACTSET_P_PER_E_LTM',
    'MACROSYNERGY_IMPINFM1Y_NSA',
    'FACTSET_EV_PER_EBITDA_LTM',
    'FACTSET_P_PER_E_LTM.1',
    'FACTSET_EV_PER_EBIT_LTM',
    'FACTSET_P_PER_FFO_LTM',
    'FACTSET_P_PER_Sales_LTM.1',
    'FACTSET_SP_500_Market_Value',
    'MACROSYNERGY_EQCUTLR_NSA',
    'MACROSYNERGY_EQCRELR_NSA',
    'MACROSYNERGY_IMPINFS1Y_NSA',
    'MACROSYNERGY_EQXR_NSA',
    'MACROSYNERGY_EQCMATR_NSA',
    'MACROSYNERGY_CDS02YCRYHvGDRB_NSA',
    'MACROSYNERGY_EQCFINR_NSA',
    'MACROSYNERGY_EQCCSRR_NSA',
    'MACROSYNERGY_GB03YYLD_NSA',
    'MACROSYNERGY_GB10YYLD_NSA',
]

In [ ]:
xgb_top_features = [
    'MACROSYNERGY_IMPINFS1Y_NSA',
    'neutral',
    'Initial Jobless Claims',
    'MACROSYNERGY_CDS02YCRY_VT10',
    'MACROSYNERGY_IMPINFM1Y_NSA',
    'FACTSET_EV_PER_EBITDA_LTM',
    'FACTSET_P_PER_Sales_LTM',
    'FACTSET_SP_500_Market_Value',
    'MACROSYNERGY_CDS02YSPRD_NSA',
    'score',
    'MACROSYNERGY_GB30YYLD_NSA',
    'MACROSYNERGY_CDS02YCRY_NSA',
    'FACTSET_SP_500_ROE_LTM',
    'MACROSYNERGY_GGDGDPRATIOX10_NSA',
    'FACTSET_SP_500_Net_Inc',
    'FACTSET_SP_500_EPS_LTM',
    'FACTSET_SP_500_Net_Margin_LTM',
    'FACTSET_SP_500_Pretax_Margin_LTM',
    'FACTSET_SP_500_BV_LTM',
]

In [ ]:
curr_target = ['FACTSET_SP_500_Close']

In [ ]:
selected_features = curr_target + list(set(lasso_top_features + xgb_top_features))

In [ ]:
df_2 = df[selected_features].copy()

Plot key features to visually check for stationarity

In [ ]:
for feature in selected_features:
    df[feature].plot()
    plt.title(feature)
    plt.show()

## Train FourierGNN model

Dataclass containing attributes for each model run specification

In [ ]:
@dataclass 
class ModelConfig:
    seq_len: int
    train_ratio: float
    val_ratio:float
    pre_len: int
    type: int
    embed_size: int
    hidden_size: int
    learning_rate: float
    n_epochs: int
    exponential_decay_step: int
    decay_rate: float 
    feature_size: int
    dataset: pd.DataFrame
    target_idx: int
    label: str
    batch_size: int
    validate_freq: int
    

Preparing variety of configurations to test

In [ ]:
model_configs = []

for df in [df_1, df_2]:
    for seq_len in [7, 31, 62]:
        for embed_size, hidden_size in zip([128, 256, 512], [256, 512, 1024]):
            for learning_rate in [0.0001, 0.00001, 0.000001]:
                model_configs.append(
                    ModelConfig(
                        seq_len=seq_len, 
                        train_ratio=0.8, 
                        val_ratio=0.1, 
                        pre_len=1, 
                        type=0, 
                        embed_size=embed_size, 
                        hidden_size=hidden_size, 
                        learning_rate=learning_rate, 
                        n_epochs=40, 
                        exponential_decay_step=5, 
                        decay_rate=0.1, 
                        feature_size=len(df.columns), 
                        dataset=df, 
                        target_idx=0,
                        label=f'FGN_model_{len(df.columns)}_{seq_len}_{embed_size}_{learning_rate}',
                        batch_size=32,
                        validatate_freq=1
                    )
                )


Functions to train FourierGNN models

In [ ]:
# Code from from https://github.com/aikunyi/FourierGNN
def validate(model, vali_loader, device, forecast_loss):
    model.eval()
    cnt = 0
    loss_total = 0
    preds = []
    trues = []
    for i, (x, y) in enumerate(vali_loader):
        cnt += 1
        y = y.float().to(device)
        x = x.float().to(device)
        forecast = torch.squeeze(model(x))
        y = y.permute(0, 2, 1).contiguous()
        loss = forecast_loss(forecast, y)
        loss_total += float(loss)
        forecast = forecast.detach().cpu().numpy()
        y = y.detach().cpu().numpy()
        preds.append(forecast)
        trues.append(y)
    preds = np.concatenate(preds, axis=0)
    trues = np.concatenate(trues, axis=0)
    score = evaluate(trues, preds)
    print(f'RAW : MAPE {score[0]:7.9%}; MAE {score[1]:7.9f}; RMSE {score[2]:7.9f}.')
    model.train()
    return loss_total/cnt

# Training code heavily borrows from https://github.com/aikunyi/FourierGNN
def train_model(model_config: ModelConfig):
    
    # Prepare training dataset
    train_dataset = Dataset_Capstone(
        data=model_config.dataset, 
        flag='train', 
        seq_len=model_config.seq_length, 
        pre_len=model_config.pre_length, 
        type=model_config.type, 
        train_ratio=model_config.train_ratio, 
        val_ratio=model_config.val_ratio
    )
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=model_config.batch_size,
        shuffle=True,
        num_workers=0,
        drop_last=False
    )

    # Prepare validation dataset
    val_dataset = Dataset_Capstone(
        data=model_config.dataset, 
        flag='val', 
        seq_len=model_config.seq_length, 
        pre_len=model_config.pre_length, 
        type=model_config.type, 
        train_ratio=model_config.train_ratio, 
        val_ratio=model_config.val_ratio
    )
    val_dataloader = DataLoader(
        train_dataset,
        batch_size=model_config.batch_size,
        shuffle=True,
        num_workers=0,
        drop_last=False
    )

    # Prepare FourierGNN model
    model = FGN(
        pre_length=model_config.pre_length, 
        embed_size=model_config.embed_size, 
        feature_size=model_config.feature_size, 
        seq_length=model_config.seq_length, 
        hidden_size=model_config.hidden_size
    )

    # Prepare optimizer and loss function
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    my_optim = torch.optim.RMSprop(params=model.parameters(), lr=model_config.learning_rate, eps=1e-08)
    my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=my_optim, gamma=model_config.decay_rate)
    forecast_loss = nn.MSELoss(reduction='mean').to(device)

    # Train model
    for epoch in range(model_config.n_epochs):
        epoch_start_time = time.time()
        model.train()
        loss_total = 0
        cnt = 0
        train_losses = []
        val_losses = []
        for index, (x, y) in enumerate(train_dataloader):
            cnt += 1
            y = y.float().to(device)
            x = x.float().to(device)
            forecast = model(x)
            y = y.permute(0, 2, 1).contiguous()
            loss = forecast_loss(forecast, y)
            loss.backward()
            my_optim.step()
            loss_total += float(loss)
            train_losses.append(float(loss))

        if (epoch + 1) % model_config.exponential_decay_step == 0:
            my_lr_scheduler.step()
        if (epoch + 1) % model_config.validate_freq == 0:
            val_loss = validate(model, val_dataloader, device, forecast_loss)
            val_losses.append(val_loss)

        print('| end of epoch {:3d} | time: {:5.2f}s | train_total_loss {:5.4f} | val_loss {:5.4f}'.format(
                epoch, (time.time() - epoch_start_time), loss_total / cnt, val_loss))
        
    # Plot loss curves
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='train_loss')
    plt.plot(val_losses, label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title(model_config.label)
    plt.show()

    # Save model
    with open(f'models/{model_config.label}', 'wb') as f:
        torch.save(model, f)


In [ ]:
for config in model_configs:
    train_model(config)
    break

Show model predictions

In [16]:
def show_model_predictions(model_config: ModelConfig):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    with open(f'models/{model_config.label}', 'rb') as f:
        model = torch.load(f)

    for label in ['train', 'val', 'test']:

        dataset = Dataset_Capstone(
            data=model_config.dataset, 
            flag=label, 
            seq_len=model_config.seq_length, 
            pre_len=model_config.pre_length, 
            type=model_config.type, 
            train_ratio=model_config.train_ratio, 
            val_ratio=model_config.val_ratio
        )
        dataloader = DataLoader(
            dataset,
            batch_size=model_config.batch_size,
            shuffle=True,
            num_workers=0,
            drop_last=False
        )

        model.eval()
        preds = []
        trues = []
        for index, (x, y) in enumerate(dataloader):
            y = y.float().to(device)
            x = x.float().to(device)
            forecast = model(x)
            y = y.permute(0, 2, 1).contiguous()
            forecast = forecast.detach().cpu().numpy()
            y = y.detach().cpu().numpy()
            preds.append(forecast)
            trues.append(y)

        preds = np.concatenate(preds, axis=0)
        trues = np.concatenate(trues, axis=0)

        # Compute performance metrics
        mse = np.mean((preds - trues) ** 2)
        directionally_correct = np.mean(np.sign(preds) == np.sign(trues))

        # Plot predictions
        plt.figure(figsize=(10, 6))
        plt.plot(preds.T[model_config.target_idx], label='Predictions')
        plt.plot(trues.T[model_config.target_idx], label='Actual')
        plt.title(f'{model_config.label} {label} : MSE {mse} : Directionally correct {directionally_correct}')
        plt.legend()
        plt.show()

        

Prepare training data for Fully Connected NN

In [ ]:
fc_y_train = fgnn_trues_train.T[0][0].reshape(-1, 1)
fc_X_train = fgnn_preds_train.T[0][:].squeeze().T
fc_X_train = np.concatenate([fc_X_train[1:], fc_y_train[:-1]], axis=1)
fc_y_train = fc_y_train[1:]
print(f'fc_X_train.shape: {fc_X_train.shape}, fc_y_train.shape: {fc_y_train.shape}')

In [12]:
tensor_fc_X_train = torch.Tensor(fc_X_train)
tensor_fc_y_train = torch.Tensor(fc_y_train)

fc_train_dataset = TensorDataset(tensor_fc_X_train, tensor_fc_y_train)
fc_train_dataloader = DataLoader(fc_train_dataset)

Specify Fully Connected NN architecture

In [16]:
class FCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(37, 120)
        self.d1 = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(120, 84)
        self.d2 = nn.Dropout(p=0.2)
        self.fc3 = nn.Linear(84, 10)
        self.fc4 = nn.Linear(10, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.d1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.d2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [ ]:
fc_model = FCNN()
fc_model

Train FCNN model

In [ ]:
fc_criterion = nn.MSELoss()
fc_learning_rate = 0.000001
fc_optimizer = optim.Adam(fc_model.parameters(), lr=fc_learning_rate)
fc_n_epochs = 100

for fc_epoch in range(fc_n_epochs):
    fc_running_loss = 0.0
    for i, data in enumerate(fc_train_dataloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        fc_optimizer.zero_grad()
        outputs = fc_model(inputs)
        loss = fc_criterion(outputs, labels)
        loss.backward()
        fc_optimizer.step()

        fc_running_loss += loss.item()
        if i % 200 == 199:
            print(f'[{fc_epoch + 1}, {i + 1:5d}] loss: {fc_running_loss / 200:}')
            fc_running_loss = 0.0

save_model(fc_model, 'output/capstone/fcnn/train', fc_epoch)

In [ ]:
fc_y_val = fgnn_trues_val.T[0][0].reshape(-1, 1)
fc_X_val = fgnn_preds_val.T[0][:].squeeze().T
fc_X_val = np.concatenate([fc_X_val[1:], fc_y_val[:-1]], axis=1)
fc_y_val = fc_y_val[1:]
print(f'fc_X_val.shape: {fc_X_val.shape}, fc_y_val.shape: {fc_y_val.shape}')

In [22]:
tensor_fc_X_val = torch.Tensor(fc_X_val)
tensor_fc_y_val = torch.Tensor(fc_y_val)

fc_val_dataset = TensorDataset(tensor_fc_X_val, tensor_fc_y_val)
fc_val_dataloader = DataLoader(fc_val_dataset)

Specify the FCNN model version to use for prediction

In [23]:
fc_model_ver = 99

Predict using FCNN model

In [ ]:
fcnn_model_path = 'output/capstone/fcnn/train'
fcnn_model = load_model(fcnn_model_path, fc_model_ver)
fcnn_model.eval()
fcnn_preds_val = []
fcnn_trues_val = []
for index, (x, y) in enumerate(fc_val_dataloader):
    y = y.float().to(device)
    x = x.float().to(device)
    fcnn_forecast_val = fcnn_model(x)
    fcnn_forecast_val = fcnn_forecast_val.detach().cpu().numpy()
    y = y.detach().cpu().numpy()
    fcnn_preds_val.append(fcnn_forecast_val)
    fcnn_trues_val.append(y)

fcnn_preds_val = np.concatenate(fcnn_preds_val, axis=0)
fcnn_trues_val = np.concatenate(fcnn_trues_val, axis=0)
fcnn_score_val = evaluate(fcnn_trues_val, fcnn_preds_val)
print(f'RAW : MAPE {fcnn_score_val[0]:7.9%}; MAE {fcnn_score_val[1]:7.9f}; RMSE {fcnn_score_val[2]:7.9f}.')

In [ ]:
plt.figure(figsize=(10, 6))

mse = np.mean((fcnn_preds_val - fcnn_trues_val)**2)
direction = np.sign(fcnn_preds_val) == np.sign(fcnn_trues_val)
direction_perc = np.sum(direction) / len(direction)

plt.plot(fcnn_preds_val, label='FCNN preds')
plt.plot(fcnn_trues_val, label='Actual')
plt.title(f'FGNN + FCNN predictions : MSE {mse:.6f} : Directionally correct {direction_perc:.1%}')
plt.legend()

Prepare test data for FCNN model

In [25]:
fc_y_test = fgnn_trues_test.T[-1][0].reshape(-1, 1)
fc_X_test = fgnn_preds_test.T[-1][:].squeeze().T
fc_X_test = np.concatenate([fc_X_test[1:], fc_y_test[:-1]], axis=1)
fc_y_test = fc_y_test[1:]
print(f'fc_X_test.shape: {fc_X_test.shape}, fc_y_test.shape: {fc_y_test.shape}')

In [26]:
tensor_fc_X_test = torch.Tensor(fc_X_test)
tensor_fc_y_test = torch.Tensor(fc_y_test)

fc_test_dataset = TensorDataset(tensor_fc_X_test, tensor_fc_y_test)
fc_test_dataloader = DataLoader(fc_test_dataset)

Specify the FCNN version to use for prediction

In [27]:
fc_model_ver = 99

Predict using FCNN model

In [ ]:
fcnn_model_path = 'output/capstone/fcnn/train'
fcnn_model = load_model(fcnn_model_path, fc_model_ver)
fcnn_model.eval()
fcnn_preds_test = []
fcnn_trues_test = []
for index, (x, y) in enumerate(fc_test_dataloader):
    y = y.float().to(device)
    x = x.float().to(device)
    fcnn_forecast_test = fcnn_model(x)
    fcnn_forecast_test = fcnn_forecast_test.detach().cpu().numpy()
    y = y.detach().cpu().numpy()
    fcnn_preds_test.append(fcnn_forecast_test)
    fcnn_trues_test.append(y)

fcnn_preds_test = np.concatenate(fcnn_preds_test, axis=0)
fcnn_trues_test = np.concatenate(fcnn_trues_test, axis=0)
fcnn_score_test = evaluate(fcnn_trues_test, fcnn_preds_test)
print(f'RAW : MAPE {fcnn_score_test[0]:7.9%}; MAE {fcnn_score_test[1]:7.9f}; RMSE {fcnn_score_test[2]:7.9f}.')